In [1]:
import pandas as pd
import jieba
import string

from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
df = pd.read_csv('category_data\cat_train_v2.csv')
df = df[['name', 'product']]
'''已知問題資料: 9691, 法文, 日文'''

'已知問題資料: 9691, 法文, 日文'

In [3]:
df = df[df['name'] != '#NAME?']
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
# df.set_index(['index'], inplace=True)

In [4]:
with open('stop.txt', 'r', encoding='utf-8') as f:
    stop = [i.strip() for i in f.readlines()]

In [6]:
df['name'] = df['name'].str.replace('\W', '', regex=True)

In [7]:
def remove_things(text):
    text = "".join([i for i in text if i not in stop])               # 移除停止詞
    text = "".join([i for i in text if i not in string.punctuation]) # 移除標點
    text = "".join([i for i in text if i not in string.whitespace])  # 移除空格
    return text

def strQ2B(ustring):
    """把字串全形轉半形"""
    ss = []
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全形空格直接轉換
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全形字元（除空格）根據關係轉化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss.append(rstring)
    return ''.join(ss)

In [8]:
df['name'] = df['name'].apply(lambda x: x.lower())
df['name'] = df['name'].apply(lambda x: remove_things(x))
df['name'] = df['name'].apply(lambda x: strQ2B(x))

In [9]:
df = df[df['name'] != '']
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)

In [11]:
mlb = MultiLabelBinarizer()
mlb_result = mlb.fit_transform([str(df.loc[i, 'product']).split(';') for i in range(len(df))])

In [15]:
for i in mlb.classes_:
    jieba.add_word(i)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dddru\AppData\Local\Temp\jieba.cache
Loading model cost 0.950 seconds.
Prefix dict has been built successfully.


In [16]:
cut_all_false = []
cut_all_true = []
HMM_false = []
HMM_true = []
paddle_false = []
paddle_true = []
all_false = []
all_true = []

for i in range(len(df)):
    cut_all_false.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=True, use_paddle=True)))
    cut_all_true.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=False, use_paddle=False)))
    HMM_false.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=False, use_paddle=True)))
    HMM_true.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=True, use_paddle=False)))
    paddle_false.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=True, use_paddle=False)))
    paddle_true.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=False, use_paddle=True)))
    all_false.append(','.join(jieba.cut(df.name[i], cut_all=False, HMM=False, use_paddle=False)))
    all_true.append(','.join(jieba.cut(df.name[i], cut_all=True, HMM=True, use_paddle=True)))

In [17]:
jb = pd.DataFrame({'cut_all_false':cut_all_false,
                    'cut_all_true': cut_all_true,
                    'HMM_false': HMM_false,
                    'HMM_true': HMM_true,
                    'paddle_false': paddle_false,
                    'paddle_true': paddle_true,
                    'all_false': all_false,
                    'all_true': all_true})
df = pd.concat([df, jb], axis=1)
df_final = pd.concat([df.drop(['product'], axis=1), pd.DataFrame(mlb_result, columns=list(mlb.classes_))], axis=1)
df_final.to_csv('preprocessed_jieba_cat_train.csv', index=False)

In [19]:
df.iloc[7]

name               舒視ml
product            人工淚液
cut_all_false     舒視,ml
cut_all_true     舒,視,ml
HMM_false        舒,視,ml
HMM_true          舒視,ml
paddle_false     舒,視,ml
paddle_true      舒,視,ml
all_false        舒,視,ml
all_true         舒,視,ml
Name: 7, dtype: object